In [ ]:
import pandas as pd
import numpy as np
import pipeline

import geopandas as gpd

import holoviews as hv, hvplot.pandas, panel as pn

pd.set_option("display.max_columns", 100)

In [ ]:
date = "2023-05-01"

In [ ]:
ais = pipeline.build_ais(date)

In [ ]:
sat_tiles = pipeline.build_tiles(date)

In [ ]:
chips = pipeline.build_chips(ais, sat_tiles)